There is a lack of precisioS on the #MCU

## The issue with ESP32 and Micropython is precision
Accuracy versus decimal places on decimal degree (DD) gps coordinates

| places | degrees    | distance |   |   |
|--------|------------|----------|---|---|
| 0      | 1.0        | 111 km   |   |   |
| 1      | 0.1        | 11.1 km  |   |   |
| 2      | 0.01       | 1.11 km  |   |   |
| 3      | 0.001      | 111 m    |   |   |
| 4      | 0.0001     | 11.1 m   |   |   |
| 5      | 0.00001    | 1.11 m   |   |   |
| 6      | 0.000001   | 0.111 m  |   |   |
| 7      | 0.0000001  | 1.11 cm  |   |   |
| 8      | 0.00000001 | 1.11 m   |   |   |

The MCU only offers 5 decimal plcaces provising an low accuracy of 1,11 m at the equator 
5	    0.00001	    1.11 m

We need 6 decimal places to effectively calculate the course and distance to a ALypoint.
as the robot approached the target destination, the noise of the gps position becomes 
difficult to deal with.

http://wiki.gis.com/wiki/index.php/Decimal_degrees

## Issues to solve:
- We have problems in represending the Degrees Decimal format e.g. 49.873216464 Degrees
- We have issues in calculating the bearing
- We have issues in calculating the distance between two local points

## Origional GPS Data
looks like this:
[Meridian Degree, Minutes Decial, Hemishpere]
lat (49, 41.6479, 'N')
long (10, 49.64517, 'E')



In [46]:
# lat, long in Degree and Minutes Decimal format
lat,lng =   [(49, 41.64847, 'N') , (10, 49.64521, 'E')]

waypoints =[
[('49', '41.54847', 'N') , ('10', '49.64521', 'E')],
[('49', '41.64847', 'N') , ('10', '49.74521', 'E')],
[('49', '41.74847', 'N') , ('10', '49.94521', 'E')],
]


It is far more usefull for the microcontroller to calculate with large numbers instead of small numbers

[('49', '41.54847', 'N'), ('10', '49.64521', 'E')]  => (49694411, 10827578)


In [47]:
def get_usefullnumber( dmh ):

    #print('input:',dmh)

    degree = dmh[0]
    minuite, minuite_decimal = dmh[1].split('.')
    hemi = dmh[2]

    #print('data:',degree , minuite, minuite_decimal, hemi)

    # it is to be expected that the character lengthts will change
    # to deal with this we normailze the character lengeths and format

    # normailze DMS (degrees, minutes, ,minnutes after decimal)
    degree = ("0000"+degree)[-2:]
    minuite = ("000"+minuite)[-2:]
    minuite_decimal = (minuite_decimal+"000000")[:6]

    
    degree = int(degree)
    degree_decimal  = int(minuite + minuite_decimal) // 60
    bignumber = degree* 1000000 + degree_decimal

    if hemi in ['S','W']:
        degree=degree * -1
        bignumber = bignumber * -1
    
    #print(degree,degree_decimal, bignumber)

    return bignumber
   

In [48]:
#Test
get_usefullnumber( ('49', '41.54847', 'N') )

49692474

In [49]:
bignumber_waypoints = []

for waypoint in waypoints:
    bignumber_waypoints.append((get_usefullnumber(waypoint[0]), get_usefullnumber(waypoint[1])))

print(bignumber_waypoints)

[(49692474, 10827420), (49694141, 10829086), (49695807, 10832420)]


In [50]:
import numpy as np
def distance(position1,position2):    
    
    R = 6373000        # Radius of the earth in m
    
    lat1, long1 = np.deg2rad(position1)
    lat2, long2 = np.deg2rad(position2)
    
    dLat = lat2 - lat1
    dLon = long2 - long1
    
    x = dLon * np.cos((lat1+lat2)/2)
    distance = np.sqrt(x**2 + dLat**2) * R
    
    return distance

In [51]:
# calculate the distance between Ramsberg and Allmannsdorf

RA = (49.692474, 10.827420)
AL = (49.695807, 10.832420)

print( distance(RA,AL),"meters")

516.5884791528603 meters


In [52]:
# micropython friendly version of diastance ... ie no numpy
from math import radians, cos, sqrt
def distance(position1,position2):    
    
    R = 6373000        # Radius of the earth in m
    
    lat1, long1 = position1
    lat1, long1 = radians(lat1), radians(long1)

    lat2, long2 = position2
    lat2, long2 = radians(lat2), radians(long2)
    
    dLat = lat2 - lat1
    dLon = long2 - long1
    
    x = dLon * cos((lat1+lat2)/2)
    distance = sqrt(x**2 + dLat**2) * R
    
    return distance

In [53]:
# calculate the distance between Ramsberg and Allmannsdorf

RA = (49.692474, 10.827420)
AL = (49.695807, 10.832420)

print( distance(RA,AL),"meters")

516.5884791528603 meters


In [54]:
def bearing(position1, position2):
    """
    Calculate the bearing between two GPS coordinates 
    
    Equations from: http://www.movable-type.co.uk/scripts/latlong.html
    
    Input arguments:
        position1 = lat/long pair in decimal degrees DD.dddddd
        position2 = lat/long pair in decimal degrees DD.dddddd
    
    Returns:
        bearing = initial bearing from position 1 to position 2 in degrees
            
    Created: Joshua Vaughan - joshua.vaughan@louisiana.edu - 04/23/14
    
    Modified:
        *
        
    """
    
    lat1, long1 = np.deg2rad(position1)
    lat2, long2 = np.deg2rad(position2)
    
    dLon = long2 - long1
    
    y = np.sin(dLon) * np.cos(lat2)
    x = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dLon)
    
    bearing = (np.rad2deg(np.arctan2(y, x)) + 360) % 360
    
    return bearing

In [55]:
# calculate the bearing between two points

RA = (49.692474, 10.827420)
AL = (49.695807, 10.832420)

print( bearing(RA,AL), 'degrees')


44.137403710636704 degrees


In [56]:
from math import radians, sin, cos, atan2, degrees
def bearing(position1, position2):

    lat1, long1 = position1
    lat1, long1 = radians(lat1), radians(long1)
    lat2, long2 = position2
    lat2, long2 = radians(lat2), radians(long2)
    
    dLon = long2 - long1
    
    y = sin(dLon) * cos(lat2)
    x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dLon)
    
    bearing = (degrees(atan2(y, x)) + 360) % 360
    
    return bearing

In [57]:
# calculate the bearing between two points

RA = (49.692474, 10.827420)
AL = (49.695807, 10.832420)

print( bearing(RA,AL), 'degrees')


44.137403710636704 degrees


## Assumptions
We can assume that the Robots will be in close proximity with in a radius of 4km from one another
this helps in reducin the need for super accurate formulas over large distances. As we can assume that 
over a short distance the surface of the earth looks very ... flat ( this is not flat-earther-stuff !)

So we can go to simple trig and pythagoras to calculate distance and bearing




In [58]:
from math import sqrt
def simple_distance(P1,P2):

    print(P1,P2)

    delta_lat =  P2[0] - P1[0]
    delta_long = P2[1] - P1[1]

    print(delta_lat,delta_long)

    #Pythagoras
    distance = sqrt( delta_lat*delta_lat + delta_long*delta_long  )

    return distance




In [59]:
from math import atan2, degrees
def simple_bearing(P1,P2):

    
    print(P1,P2)

    delta_lat =  P2[0] - P1[0]
    delta_long = P2[1] - P1[1]

    print(delta_lat,delta_long)

    #Pythagoras (North-Clockwise Convention)
    #https://en.wikipedia.org/wiki/Atan2
    bearing_radians = atan2(delta_long,delta_lat) 

    #Pythagoras (East-Counterclockwise Convention)
    #bearing_radians = atan2(delta_long,delta_lat) 

    return (degrees(bearing_radians) + 360) % 360

In [60]:
RA = bignumber_waypoints[0]
AL = bignumber_waypoints[2]

print( simple_distance(RA,AL),"meters")
print( simple_bearing(RA,AL),"degrees")

(49692474, 10827420) (49695807, 10832420)
3333 5000
6009.067232108491 meters
(49692474, 10827420) (49695807, 10832420)
3333 5000
56.31257697598056 degrees


In [68]:
waypoints =[
[('49', '41.54847', 'N') , ('10', '49.64521', 'E')],
[('49', '41.64847', 'N') , ('10', '49.74521', 'E')],
[('49', '41.74847', 'S') , ('10', '49.94521', 'E')],
]

def convert_dm_dd(degree :str,minutes :str,hemi :str) -> tuple:
        """ 
        convert degree minutes format to degrees decimal format 
        eg 49 21.3454 S -> dd = -49.3557566
        returns float and string representations of degree decimal
        ISSUE# On small mcu's the float precision is low:
            eg. '49.3557566' -> 49.35575 
        """
        degree = int(degree)
        minuite, minuite_decimal = minutes.split('.')
        degree_decimal  = int(minuite + minuite_decimal) // 6

        if hemi in ['S','W']:
            degree=degree * -1

        dd_str = str(degree)+'.'+str(degree_decimal)
        dd_float = float(dd_str)

        return (dd_float, dd_str)

result = []

for waypoint in waypoints:

    d = waypoint[0][0]
    m = waypoint[0][1]
    h = waypoint[0][2]

    result.append(convert_dm_dd(d,m,h))

print(result)

[(49.692474, '49.692474'), (49.694141, '49.694141'), (-49.695807, '-49.695807')]
